# User Level Analysis
- 1) Split Sample Analysis using main effect (T_d x P_t)
- 2) Split Sample Analysis using DDD effect (T_d x P_t x Techiness)

### 1) Split Sample Analysis using Main effect

In [18]:
# Load Pickle
import pickle
import pandas as pd
import numpy as np

with open(file='/data1/StackOverflow/_Final/LSM_new_result.pickle', mode='rb') as f:
    lsm_og=pickle.load(f)
    
import pickle
import pandas as pd
import numpy as np
import sqlite3
# Import Users Data
conn = sqlite3.connect('/data1/StackOverflow/stackexchange-to-sqlite/stack.db')
query = '''
SELECT id, creation_date
FROM users;
'''
user_df = pd.read_sql_query(query, conn)
conn.close()

In [19]:
user_df

,id,creation_date
0,-1016,2023-06-28 20:22:59.967
1,-1015,2023-06-28 18:50:43.493
2,-1014,2023-02-17 19:52:56.213
3,-1013,2023-02-17 19:25:19.953
4,-1012,2023-02-15 23:24:13.363
...,...,...
21077275,22492432,2023-09-03 11:44:44.337
21077276,22492433,2023-09-03 11:44:49.580
21077277,22492434,2023-09-03 11:44:57.507
21077278,22492435,2023-09-03 11:45:03.880


In [25]:
# Merge
user_df = user_df.rename(columns = {'creation_date':'account_creation_date',
                                    'id': 'owner_user_id'})
user_df['owner_user_id'] = user_df.owner_user_id.astype(str)
df_merge = pd.merge(lsm_og, user_df, on='owner_user_id', how = 'left')
df_merge

,owner_user_id,year_month,body,lsm_score,account_creation_date
0,10000229,2021-09-01,<p>One thing that I've noticed in your code is...,0.157493,2018-06-27 13:19:58.483
1,10002945,2021-09-01,<p>I am sure the above answer is very good and...,0.162539,2018-06-28 01:40:19.807
2,10008173,2021-09-01,<p>Since you already know the location on the ...,0.341333,2018-06-28 22:58:51.250
3,10008643,2021-09-01,<p>Replace &quot;justify-content-center&quot; ...,0.047047,2018-06-29 02:46:04.963
4,10009777,2021-09-01,<p>(might need to adjust syntax according to y...,0.063511,2018-06-29 08:18:44.570
...,...,...,...,...,...
2240706,9973516,2023-08-31,<p>Your <code>bf</code> struct contains only t...,0.203579,2018-06-21 13:53:17.230
2240707,997358,2023-08-31,<p>A simple solution is readily obtained if th...,0.094842,2011-02-24 21:55:46.377
2240708,997378,2023-08-31,<p>Use <code>!curl ipinfo.io</code> to check w...,0.079271,2011-10-15 23:45:47.393
2240709,9977815,2023-08-31,<p>I finally figured it out.\nI had to use</p>...,0.068769,2018-06-22 10:34:07.510


In [26]:
# divide the data into posts before and after 2020-09-01
df_old = df_merge[df_merge['account_creation_date'] < '2020-09-01']
df_new = df_merge[df_merge['account_creation_date'] >= '2020-09-01']

In [27]:
print(len(df_old))
print(len(df_new))

1633573
607138


In [30]:
# Aggregation (Split old and new)
lsm_old = df_old.groupby(['year_month'])['lsm_score'].mean().reset_index()
lsm_new = df_new.groupby(['year_month'])['lsm_score'].mean().reset_index()
lsm_old

,year_month,lsm_score
0,2021-09-01,0.103758
1,2021-09-02,0.117477
2,2021-09-03,0.111368
3,2021-09-04,0.128617
4,2021-09-05,0.131078
...,...,...
725,2023-08-27,0.140123
726,2023-08-28,0.134967
727,2023-08-29,0.123025
728,2023-08-30,0.159574


In [39]:
# import entropy data
df_old = pd.read_csv("/data1/StackOverflow/_Robustness/H4/df_old_vr2.csv")
df_new = pd.read_csv("/data1/StackOverflow/_Robustness/H4/df_new_vr2.csv")

In [40]:
df_old['lsm'] = lsm_old['lsm_score']
df_new['lsm'] = lsm_new['lsm_score']
df_old['ln_lsm'] = np.log(df_old['lsm'])
df_new['ln_lsm'] = np.log(df_new['lsm'])

In [43]:
df_old.to_csv('main_lsm_old.csv', index = False)
df_new.to_csv('main_lsm_new.csv', index = False)

### 2. Split Sample Analysis using DDD Effect

In [45]:
# import og data (includes entropy)
DDD_old = pd.read_csv('/data1/StackOverflow/_Robustness/H4/DDD/df_final_old_vr2.csv')
DDD_new = pd.read_csv('/data1/StackOverflow/_Robustness/H4/DDD/df_final_new_vr2.csv')

In [46]:
DDD_new

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,...,ln_entropy,year_month,numUser,ln_numUser,comments,CommentsPerP,AnsPerQ,ln_comments,ln_AnsPerQ,ln_CommentsPerP
0,2021-09-01,0,0,9,0,0.525040,6.588325,463,200,6.137727,...,1.885299,2021-09,8865,9.089866,1375,1.576835,1.129590,7.226209,0.121854,0.455420
1,2021-09-01,0,0,9,1,0.412791,6.426070,129,42,4.859812,...,1.860363,2021-09,2574,7.853216,467,2.213270,1.023256,6.146329,0.022990,0.794471
2,2021-09-01,0,0,9,3,0.490224,6.357754,144,32,4.969813,...,1.849675,2021-09,2900,7.972466,441,1.792683,1.097222,6.089045,0.092782,0.583713
3,2021-09-01,0,0,9,4,0.439161,6.637747,126,37,4.836282,...,1.892773,2021-09,3118,8.044947,738,3.024590,1.182540,6.603944,0.167664,1.106776
4,2021-09-01,0,0,9,5,0.461350,6.374922,138,45,4.927254,...,1.852372,2021-09,2629,7.874359,376,1.649123,0.971014,5.929589,-0.029414,0.500244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9746,2023-08-31,1,1,8,9,0.541652,3.980927,13,3,2.564949,...,1.381515,2023-08,310,5.736572,27,1.350000,0.615385,3.295837,-0.485508,0.300105
9747,2023-08-31,1,1,8,11,0.476285,3.286896,7,2,1.945910,...,1.189944,2023-08,138,4.927254,15,1.666667,0.285714,2.708050,-1.252763,0.510826
9748,2023-08-31,1,1,8,13,0.421145,3.794653,8,7,2.079442,...,1.333593,2023-08,237,5.468060,9,0.750000,0.500000,2.197225,-0.693147,-0.287682
9749,2023-08-31,1,1,8,15,0.307775,3.000000,4,2,1.386294,...,1.098612,2023-08,54,3.988984,6,1.000000,0.750000,1.791759,-0.287682,0.000000


In [47]:
# Load Pickle of newly computed LSM (daily aggregation)
import pickle
import pandas as pd
import numpy as np

with open(file='/data1/StackOverflow/_Robustness/Techiness/_LSM_Community/LSM_new_community_result.pickle', mode='rb') as f:
    lsm_result=pickle.load(f)
lsm_result

,owner_user_id,year_month,body,community,lsm_score
0,3027266,2021-09-01,"<p>Try the <a href=""https://momentjs.com/docs/...",0,0.140352
1,13981886,2021-09-01,<p>Change to this</p>\n<pre><code>:class=&quot...,0,0.143264
2,892629,2021-09-01,<p>This is a rounding error with floating poin...,0,0.347610
3,6010118,2021-09-01,<p>I was able to fix this by setting mocha's <...,0,0.177443
4,6009931,2021-09-01,<p>Turns out that Burpsuite has a free version...,0,0.136490
...,...,...,...,...,...
2237154,17507911,2023-08-31,"<pre class=""lang-py prettyprint-override""><cod...",17,0.156018
2237155,11759438,2023-08-31,<p>The message is telling you to check <code>v...,17,0.171527
2237156,16647496,2023-08-31,<p>That is controlled by the <code>labels</cod...,17,0.174017
2237157,1919793,2023-08-31,"<p>In addition to <a href=""https://stackoverfl...",17,0.153045


In [51]:
# Merge
user_df['owner_user_id'] = user_df.owner_user_id.astype(str)
df_merge = pd.merge(lsm_result, user_df, on='owner_user_id', how = 'left')
df_merge

,owner_user_id,year_month,body,community,lsm_score,account_creation_date
0,3027266,2021-09-01,"<p>Try the <a href=""https://momentjs.com/docs/...",0,0.140352,2013-11-24 12:58:35.510
1,13981886,2021-09-01,<p>Change to this</p>\n<pre><code>:class=&quot...,0,0.143264,2020-07-23 10:24:22.153
2,892629,2021-09-01,<p>This is a rounding error with floating poin...,0,0.347610,2011-08-13 00:50:21.397
3,6010118,2021-09-01,<p>I was able to fix this by setting mocha's <...,0,0.177443,2016-03-02 21:25:48.923
4,6009931,2021-09-01,<p>Turns out that Burpsuite has a free version...,0,0.136490,2016-03-02 20:34:28.000
...,...,...,...,...,...,...
2237154,17507911,2023-08-31,"<pre class=""lang-py prettyprint-override""><cod...",17,0.156018,2021-11-25 12:56:39.797
2237155,11759438,2023-08-31,<p>The message is telling you to check <code>v...,17,0.171527,2019-07-09 11:45:59.647
2237156,16647496,2023-08-31,<p>That is controlled by the <code>labels</cod...,17,0.174017,2021-08-12 06:52:34.630
2237157,1919793,2023-08-31,"<p>In addition to <a href=""https://stackoverfl...",17,0.153045,2012-12-20 18:58:57.833


In [54]:
# divide the data into posts before and after 2020-09-01
df_old_DDD = df_merge[df_merge['account_creation_date'] < '2020-09-01']
df_new_DDD = df_merge[df_merge['account_creation_date'] >= '2020-09-01']

In [65]:
print("[Old DB] # of Answers: ", len(df_old_DDD), "# of Users: ", df_old_DDD.owner_user_id.nunique())
print("[New DB] # of Answers: ", len(df_new_DDD), "# of Users: ", df_new_DDD.owner_user_id.nunique())

[Old DB] # of Answers:  1631897 # of Users:  433009
[New DB] # of Answers:  605262 # of Users:  284261


In [74]:
# Aggregation (Split old and new)
lsm_old = df_old_DDD.groupby(['year_month', 'community'])['lsm_score'].mean().reset_index()
lsm_new = df_old_DDD.groupby(['year_month', 'community'])['lsm_score'].mean().reset_index()
lsm_old = lsm_old.rename(columns={'year_month':'year_month_day'})
lsm_new = lsm_new.rename(columns={'year_month':'year_month_day'})

In [75]:
lsm_old

,year_month_day,community,lsm_score
0,2021-09-01,0,0.185265
1,2021-09-01,1,0.216306
2,2021-09-01,3,0.229911
3,2021-09-01,4,0.187508
4,2021-09-01,5,0.211745
...,...,...,...
10873,2023-08-31,10,1.000000
10874,2023-08-31,11,0.628668
10875,2023-08-31,13,0.497886
10876,2023-08-31,15,0.765648


In [73]:
DDD_old

,year_month_day,T_d,P_t,month,community,techiness,entropy,count_q,count_a,ln_q,...,ln_entropy,year_month,numUser,ln_numUser,comments,CommentsPerP,AnsPerQ,ln_comments,ln_AnsPerQ,ln_CommentsPerP
0,2021-09-01,0,0,9,0,0.525040,7.320204,727,814,6.588926,...,1.990638,2021-09,13451,9.506809,2163,1.637396,1.181568,7.679251,0.166842,0.493107
1,2021-09-01,0,0,9,1,0.412791,7.269554,269,182,5.594711,...,1.983695,2021-09,5246,8.565221,814,1.975728,1.040892,6.701960,0.040078,0.680937
2,2021-09-01,0,0,9,3,0.490224,7.115008,243,215,5.493061,...,1.962206,2021-09,4814,8.479284,676,1.751295,0.962963,6.516193,-0.037740,0.560356
3,2021-09-01,0,0,9,4,0.439161,7.490236,230,268,5.438079,...,2.013600,2021-09,4602,8.434246,1051,2.582310,1.130435,6.957497,0.122602,0.948684
4,2021-09-01,0,0,9,5,0.461350,6.844602,236,213,5.463832,...,1.923460,2021-09,4880,8.492900,588,1.466334,1.326271,6.376727,0.282371,0.382766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264,2023-08-31,1,1,8,9,0.541652,4.381050,18,13,2.890372,...,1.477288,2023-08,200,5.298317,27,1.000000,0.611111,3.295837,-0.492476,0.000000
10265,2023-08-31,1,1,8,10,0.373225,2.947703,3,1,1.098612,...,1.081026,2023-08,45,3.806662,1,0.250000,0.333333,0.000000,-1.098612,-1.386294
10266,2023-08-31,1,1,8,11,0.476285,3.708966,11,10,2.397895,...,1.310753,2023-08,205,5.323010,15,0.714286,0.909091,2.708050,-0.095310,-0.336472
10267,2023-08-31,1,1,8,13,0.421145,5.317946,21,11,3.044522,...,1.671087,2023-08,393,5.973810,40,1.142857,0.380952,3.688879,-0.965081,0.133531


In [79]:
lsm_old['year_month_day'] = lsm_old['year_month_day'].astype(str)
lsm_new['year_month_day'] = lsm_new['year_month_day'].astype(str)

In [80]:
DDD_old = pd.merge(DDD_old, lsm_old, on = ['year_month_day', 'community'], how = 'left')
DDD_new = pd.merge(DDD_new, lsm_new, on = ['year_month_day', 'community'], how = 'left')

In [85]:
DDD_old = DDD_old.rename(columns = {'lsm_score':'lsm'})
DDD_new = DDD_new.rename(columns = {'lsm_score':'lsm'})
DDD_old['ln_lsm'] = np.log(DDD_old['lsm'])
DDD_new['ln_lsm'] = np.log(DDD_new['lsm'])

In [86]:
DDD_old.to_csv('DDD_lsm_old.csv', index = False)
DDD_new.to_csv('DDD_lsm_new.csv', index = False)